### **Importando bibliotecas**

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Dense, Input
from keras import backend as K

import numpy as np

### **"Arrumando" o dataset para o treinamento**

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12

# Dimensões das imagens
img_rows, img_cols = 28, 28

# Carrega o dataset do mnist, embaralha os dados
# e separa em conjunto de treinamento e teste
(x_train, y_train), (x_test, y_test) = mnist.load_data()
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normaliza os dados para serem 0 ou 1
# Fiz um teste sem normalizar e não senti diferença para esse dataset
x_train /= 255
x_test /= 255

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converte os alvos para serem vetores 10x1 em que somente o neurônio
# corresponde ao alvo é ativado.
# Ex: Entrada é o número 5 -> Elemento 5 do vetor de saída vale "1" e os outros "0"
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### **Montagem da rede e pré-treinamento visando reconstrução**

In [ ]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# No SAE, o alvo é a própria entrada
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

### **Treinamento supervisionado para classificação**

In [ ]:
nn = Sequential ()
for layer in range (1, (len(autoencoder.layers)/2 + 1)):
    nn.add (autoencoder.layers[layer])
    
nn.add(Dense(num_classes, activation='softmax'))
     
nn.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

nn.fit(x_train, y_train,
        epochs=10,
        batch_size=256,
        shuffle=True,
        validation_data=(x_test, y_test))
 
score = nn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### **Ilustração da reconstrução feita pelo SAE**

In [ ]:
import matplotlib.pyplot as plt

decoded_imgs = autoencoder.predict (x_test)

n = 10  # Quantos digitos vão ser mostrados
plt.figure(figsize=(20, 4))
for i in range(n):
    # Imagem original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Image reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()